In [1]:
import os
import sqlite3
import pandas as pd
from fuzzywuzzy import process

In [2]:
conn = sqlite3.connect("../data/db/NBA-L2M.db")
c = conn.cursor()

In [23]:
reports = pd.read_sql("SELECT * FROM reports", conn)
reports.replace({'away':{'PHO': 'PHX'}})
reports.head(2)
reports.columns

Index([u'report', u'away', u'home', u'date', u'season', u'away_score',
       u'home_score', u'winner'],
      dtype='object')

In [4]:
calls = pd.read_sql("SELECT * FROM calls", conn)
calls.head(2)

,report,period,time,call,committing,disadvantaged,decision,committing_team,disadvantaged_team
0,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,CC,NaN,NaN
1,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Karl-Anthony Towns,Danilo Gallinari,CNC,NaN,NaN


In [5]:
def players_dict(c):
    players_year = {}
    for f in os.listdir("../data/csv/"):
        players = c.execute("""SELECT Tm, Player FROM '{}'""".format(f[:13])).fetchall()
        year = {}
        for player in players:
            if player[0] not in year:
                year[player[0]] = []
            year[player[0]].append(player[1])
        players_year[f[8:13]] = year
    return players_year

players = players_dict(c)

In [6]:
df = pd.merge(calls, reports, how='left', on='report', )
df.head(2)

,report,period,time,call,committing,disadvantaged,decision,committing_team,disadvantaged_team,away,home,date,season,away_score,home_score,winner
0,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,CC,NaN,NaN,DEN,MIN,2016-11-02 00:00:00,2016-17 NBA Regular Season,NaN,NaN,NaN
1,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Karl-Anthony Towns,Danilo Gallinari,CNC,NaN,NaN,DEN,MIN,2016-11-02 00:00:00,2016-17 NBA Regular Season,NaN,NaN,NaN


In [7]:
for row in df.iterrows():
    year = row[1].season[2:7]
    away = players[year][row[1].away]
    home = players[year][row[1].home]
    
    if row[1].committing != '':
        _, c_away = process.extractOne(row[1].committing, away)
        _, c_home = process.extractOne(row[1].committing, home)
    else:
        c_away = 0
        c_home = 0
    
    if row[1].disadvantaged != '':
        _, d_away = process.extractOne(row[1].disadvantaged, away)
        _, d_home = process.extractOne(row[1].disadvantaged, home)
    else:
        d_away = 0
        d_home = 0
        
    if max(c_away, c_home) < 50:
        row[1].committing_team = "NaN"
    elif c_away > c_home:
        row[1].committing_team = row[1].away
    else:
        row[1].committing_team = row[1].home
    
    if max(d_away, d_home) < 50:
        row[1].disadvantaged_team = "NaN"
    elif d_away > d_home:
        row[1].disadvantaged_team = row[1].away
    else:
        row[1].disadvantaged_team = row[1].home    

Karl-Anthony Towns MIN Emmanuel Mudiay DEN
Karl-Anthony Towns MIN Danilo Gallinari DEN
Emmanuel Mudiay DEN  NaN
Karl-Anthony Towns MIN Emmanuel Mudiay DEN
Jusuf Nurkic DEN Nemanja Bjelica MIN
 NaN  NaN
Tyus Jones MIN Jameer Nelson DEN
Karl-Anthony Towns MIN Jusuf Nurkic DEN
Karl-Anthony Towns MIN  NaN
Nemanja Bjelica MIN Jameer Nelson DEN
 NaN  NaN
Danilo Gallinari DEN Karl-Anthony Towns MIN
Dwyane Wade CHI Garrett Temple SAC
Kosta Koufos SAC Dwyane Wade CHI
Taj Gibson CHI Garrett Temple SAC
Kosta Koufos SAC Robin Lopez CHI
 NaN  NaN
Taj Gibson CHI  NaN
Robin Lopez CHI  NaN
Arron Afflalo SAC Jimmy Butler CHI
Taj Gibson CHI DeMarcus Cousins SAC
DeMarcus Cousins SAC  NaN
Garrett Temple SAC Taj Gibson CHI
Paul Zipser CHI DeMarcus Cousins SAC
Jimmy Butler CHI Garrett Temple SAC
Kosta Koufos SAC Taj Gibson CHI
Kosta Koufos SAC Jimmy Butler CHI
Dwyane Wade CHI Ty Lawson SAC
Dwyane Wade CHI Garrett Temple SAC
 NaN  NaN
DeMarcus Cousins SAC Dwyane Wade CHI
Taj Gibson CHI DeMarcus Cousins SAC
D

Kent Bazemore ATL Evan Fournier ORL
Nikola Vucevic ORL Kent Bazemore ATL
 NaN  NaN
Cole Aldrich LAC Rudy Gay SAC
DeMarcus Cousins SAC Chris Paul LAC
Darren Collison SAC Chris Paul LAC
Rudy Gay SAC  NaN
Paul Pierce LAC Darren Collison SAC
Austin Rivers LAC Rudy Gay SAC
JJ Redick LAC Darren Collison SAC
Chris Paul LAC Rajon Rondo SAC
Jamal Crawford LAC Willie Cauley-Stein SAC
Udonis Haslem MIA LaMarcus Aldridge POR
Hassan Whiteside MIA Robin Lopez POR
Udonis Haslem MIA Nicolas Batum POR
, NaN  NaN
CJ McCollum POR Dwyane Wade MIA
Buddy Hield NOP Thabo Sefolosha ATL
Dennis Schroder ATL Buddy Hield NOP
Dwight Howard ATL  NaN
Dwight Howard ATL Buddy Hield NOP
Anthony Davis NOP Tim Hardaway Jr. ATL
Dennis Schroder ATL  NaN
E'Twaun Moore NOP Dennis Schroder ATL
Anthony Davis NOP Paul Millsap ATL
Paul Millsap ATL  NaN
Paul Millsap ATL  NaN
Langston Galloway NOP Paul Millsap ATL
Kent Bazemore ATL Anthony Davis NOP
E'Twaun Moore NOP Paul Millsap ATL
Reggie Jackson DET LeBron James CLE
Reggie Jack

Hassan Whiteside MIA Elfrid Payton ORL
Evan Fournier ORL Dwyane Wade MIA
Goran Dragic MIA Evan Fournier ORL
, NaN  NaN
Wilson Chandler DEN James Harden HOU
James Harden HOU Nikola Jokic DEN
Nene HOU Wilson Chandler DEN
Danilo Gallinari DEN James Harden HOU
James Harden HOU Nikola Jokic DEN
 NaN  NaN
 NaN  NaN
Gary Harris DEN James Harden HOU
 NaN  NaN
Nikola Jokic DEN Nene HOU
 NaN  NaN
Danilo Gallinari DEN James Harden HOU
Jameer Nelson DEN  NaN
Nikola Jokic DEN Patrick Beverley HOU
Patrick Beverley HOU Danilo Gallinari DEN
Wilson Chandler DEN James Harden HOU
James Harden HOU  NaN
Jameer Nelson DEN Eric Gordon HOU
Kevin Love CLE Kyle Lowry TOR
Patrick Patterson TOR Matthew Dellavedova CLE
DeMar DeRozan TOR Kevin Love CLE
Kevin Love CLE Bismack Biyombo TOR
Kyle Lowry TOR Kevin Love CLE
Jonas Valanciunas TOR Matthew Dellavedova CLE
Matthew Dellavedova CLE Kyle Lowry TOR
 NaN  NaN
 NaN  NaN
Matthew Dellavedova CLE Patrick Patterson TOR
 NaN  NaN
LeBron James CLE  NaN
Cory Joseph TOR LeB

 NaN  NaN
DeMarre Carroll TOR Rudy Gay SAC
Rudy Gay SAC DeMarre Carroll TOR
DeMarre Carroll TOR  NaN
Rudy Gay SAC Jonas Valanciunas TOR
Jonas Valanciunas TOR DeMarcus Cousins SAC
DeMarcus Cousins SAC DeMarre Carroll TOR
DeMarcus Cousins SAC DeMar DeRozan TOR
DeMar DeRozan TOR  NaN
Darren Collison SAC Kyle Lowry TOR
 NaN  NaN
 NaN  NaN
Darren Collison SAC  NaN
Kyle Lowry TOR Darren Collison SAC
 NaN  NaN
Patrick Patterson TOR Darren Collison SAC
 NaN  NaN
J.R. Smith CLE James Harden HOU
LeBron James CLE James Harden HOU
 NaN  NaN
Josh Smith HOU Kevin Love CLE
J.R. Smith CLE James Harden HOU
Trevor Ariza HOU LeBron James CLE
Terrence Jones HOU Tristan Thompson CLE
Tristan Thompson CLE Josh Smith HOU
Terrence Jones HOU LeBron James CLE
Tristan Thompson CLE Terrence Jones HOU
 NaN  NaN
LeBron James CLE James Harden HOU
LeBron James CLE James Harden HOU
Josh Smith HOU LeBron James CLE
Trevor Ariza HOU Tristan Thompson CLE
Kevin Love CLE Josh Smith HOU
James Harden HOU LeBron James CLE
LeBro

Carmelo Anthony NYK Gordon Hayward UTA
Carmelo Anthony NYK Gordon Hayward UTA
Gordon Hayward UTA Carmelo Anthony NYK
Carmelo Anthony NYK  NaN
Rodney Hood UTA Derrick Williams NYK
John Wall WAS Jameer Nelson DEN
John Wall WAS Danilo Gallinari DEN
Nikola Jokic DEN Kelly Oubre WAS
Jameer Nelson DEN Bradley Beal WAS
Wilson Chandler DEN Kelly Oubre WAS
Otto Porter WAS Nikola Jokic DEN
 NaN  NaN
Jameer Nelson DEN  NaN
Nikola Jokic DEN Bradley Beal WAS
John Wall WAS Jameer Nelson DEN
Marcin Gortat WAS Nikola Jokic DEN
Nikola Jokic DEN John Wall WAS
Nikola Jokic DEN John Wall WAS
 NaN  NaN
 NaN  NaN
Gorgui Dieng MIN  NaN
Gorgui Dieng MIN Jordan Clarkson LAL
Jordan Clarkson LAL Zach LaVine MIN
Tarik Black LAL Sean Kilpatrick MIN
Gorgui Dieng MIN Jordan Clarkson LAL
Lorenzo Brown MIN Wayne Ellington LAL
Wayne Ellington LAL Andrew Wiggins MIN
Andrew Wiggins MIN Tarik Black LAL
Zach LaVine MIN  NaN
Wayne Ellington LAL Andrew Wiggins MIN
Jordan Clarkson LAL  NaN
Wayne Ellington LAL Andrew Wiggins M

...

 UTA
Trevor Booker UTA Patrick Beverley HOU
Chris Johnson UTA James Harden HOU
Trevor Booker UTA Trevor Ariza HOU
Montrezl Harrell HOU Gordon Hayward UTA
Trey Lyles UTA Terrence Jones HOU
Raul Neto UTA Marcus Thornton HOU
Joe Ingles UTA  NaN
Montrezl Harrell HOU  NaN
Karl-Anthony Towns MIN Nikola Jokic DEN
Jameer Nelson DEN Tyus Jones MIN
Gary Harris DEN  NaN
Tyus Jones MIN Wilson Chandler DEN
Karl-Anthony Towns MIN  NaN
Danilo Gallinari DEN  NaN
Jameer Nelson DEN Tyus Jones MIN
Wilson Chandler DEN Andrew Wiggins MIN
Karl-Anthony Towns MIN  NaN
Nikola Jokic DEN Kris Dunn MIN
Tyus Jones MIN Jameer Nelson DEN
Amir Johnson TOR Otto Porter WAS
Greivis Vasquez TOR Bradley Beal WAS
, NaN  NaN
Paul Pierce WAS Greivis Vasquez TOR
Paul Pierce WAS  NaN
Paul Pierce WAS Amir Johnson TOR
Amir Johnson TOR John Wall WAS
Paul Pierce WAS  NaN
Greivis Vasquez TOR Bradley Beal WAS
DeMar DeRozan TOR Paul Pierce WAS
Paul Pierce WAS Greivis Vasquez TOR
Paul Pierce WAS DeMar DeRozan TOR
Marcin Gortat WAS DeM

NYK
Jared Sullinger BOS Robin Lopez NYK
Carmelo Anthony NYK Isaiah Thomas BOS
Robin Lopez NYK , NaN
Evan Turner BOS Robin Lopez NYK
Jae Crowder BOS Carmelo Anthony NYK
Lance Thomas NYK Avery Bradley BOS
Jae Crowder BOS Carmelo Anthony NYK
Jae Crowder BOS Carmelo Anthony NYK
Marcus Smart BOS Carmelo Anthony NYK
Marcus Smart BOS Carmelo Anthony NYK
Anthony Morrow OKC Marc Gasol MEM
Vince Carter MEM  NaN
Andre Roberson OKC Vince Carter MEM
Steven Adams OKC Marc Gasol MEM
James Ennis III MEM Russell Westbrook OKC
Zach Randolph MEM Steven Adams OKC
Vince Carter MEM Steven Adams OKC
Vince Carter MEM Andre Roberson OKC
James Ennis III MEM Russell Westbrook OKC
 NaN  NaN
Jimmy Butler CHI Danilo Gallinari DEN
 NaN  NaN
 NaN  NaN
 NaN  NaN
Jusuf Nurkic DEN Rajon Rondo CHI
Robin Lopez CHI  NaN
Jusuf Nurkic DEN Dwyane Wade CHI
Robin Lopez CHI Will Barton DEN
Will Barton DEN  NaN
Rajon Rondo CHI Danilo Gallinari DEN
Emmanuel Mudiay DEN Jimmy Butler CHI
Rajon Rondo CHI Emmanuel Mudiay DEN
Robin Lope


CJ McCollum POR Rodney Stuckey IND
Justise Winslow MIA Marvin Williams CHA
Kemba Walker CHA Luol Deng MIA
Al Jefferson CHA Joe Johnson MIA
Courtney Lee CHA Dwyane Wade MIA
Marvin Williams CHA Luol Deng MIA
Marvin Williams CHA Luol Deng MIA
Joe Johnson MIA Nicolas Batum CHA
, NaN  NaN
Al Jefferson CHA Hassan Whiteside MIA
Dwyane Wade MIA Kemba Walker CHA
Luol Deng MIA Marvin Williams CHA
Hassan Whiteside MIA Marvin Williams CHA
Courtney Lee CHA Dwyane Wade MIA
Luol Deng MIA Jeremy Lin CHA
Goran Dragic MIA Courtney Lee CHA
Goran Dragic MIA Nicolas Batum CHA
Luol Deng MIA Jeremy Lin CHA
Evan Fournier ORL Jarrett Jack BKN
Victor Oladipo ORL Jarrett Jack BKN
Brook Lopez BKN Victor Oladipo ORL
Jarrett Jack BKN Victor Oladipo ORL
Brook Lopez BKN Tobias Harris ORL
Bojan Bogdanovic BKN Aaron Gordon ORL
Brook Lopez BKN Victor Oladipo ORL
Wayne Ellington BKN Nikola Vucevic ORL
Andrew Wiggins MIN DeMar DeRozan TOR
Andrew Wiggins MIN DeMar DeRozan TOR
Patrick Patterson TOR Karl-Anthony Towns MIN
R

PJ Tucker PHX Derrick Favors UTA
 NaN  NaN
Gerald Green PHX  NaN
Rudy Gobert UTA  NaN
Dante Exum UTA Eric Bledsoe PHX
Eric Bledsoe PHX Dante Exum UTA
 NaN  NaN
Derrick Favors UTA Markieff Morris PHX
Gordon Hayward UTA Eric Bledsoe PHX
Gerald Green PHX Derrick Favors UTA
Gordon Hayward UTA Eric Bledsoe PHX
Brandan Wright PHX Joe Ingles UTA
PJ Tucker PHX Rodney Hood UTA
Ryan Kelly LAL  NaN
Brandon Bass LAL PJ Tucker PHX
Brandon Knight PHX  NaN
Tyson Chandler PHX  NaN
Larry Nance Jr. LAL Brandon Knight PHX
Tyson Chandler PHX Ryan Kelly LAL
Larry Nance Jr. LAL Devin Booker PHX
Tyson Chandler PHX Lou Williams LAL
Ryan Kelly LAL PJ Tucker PHX
Mirza Teletovic PHX Larry Nance Jr. LAL
PJ Tucker PHX Brandon Bass LAL
, NaN  NaN
Devin Booker PHX  NaN
Brandon Bass LAL Devin Booker PHX
Larry Nance Jr. LAL PJ Tucker PHX
PJ Tucker PHX Marcelo Huertas LAL
Lou Williams LAL  NaN
 NaN  NaN
Lou Williams LAL Brandon Knight PHX
Lou Williams LAL Devin Booker PHX
Pau Gasol SAS Eric Bledsoe PHX
PJ Tucker PHX Ka

Trevor Booker UTA Kevin Durant OKC
Norris Cole NOP Jordan Clarkson LAL
Julius Randle LAL , NaN
Larry Nance Jr. LAL Tyreke Evans NOP
Jrue Holiday NOP Lou Williams LAL
Larry Nance Jr. LAL Jrue Holiday NOP
Ryan Anderson NOP Lou Williams LAL
Alonzo Gee NOP Julius Randle LAL
Jrue Holiday NOP Lou Williams LAL
Ryan Anderson NOP Lou Williams LAL
Tyreke Evans NOP Jordan Clarkson LAL
Ryan Anderson NOP Julius Randle LAL
Giannis Antetokounmpo MIL  NaN
Jabari Parker MIL Wesley Matthews DAL
Giannis Antetokounmpo MIL Harrison Barnes DAL
Giannis Antetokounmpo MIL Jose Juan Barea DAL
Giannis Antetokounmpo MIL  NaN
Andrew Bogut DAL Mirza Teletovic MIL
Mirza Teletovic MIL Dorian Finney-Smith DAL
Mirza Teletovic MIL Jose Juan Barea DAL
Jose Juan Barea DAL Matthew Dellavedova MIL
Giannis Antetokounmpo MIL Jose Juan Barea DAL
 NaN  NaN
Andrew Bogut DAL  NaN
Giannis Antetokounmpo MIL Harrison Barnes DAL
Jabari Parker MIL Wesley Matthews DAL
Jabari Parker MIL Jose Juan Barea DAL
Jason Terry MIL  NaN
Jabari Pa

In [20]:
df.ix[:, :9]

,report,period,time,call,committing,disadvantaged,decision,committing_team,disadvantaged_team
0,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,CC,MIN,DEN
1,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Karl-Anthony Towns,Danilo Gallinari,CNC,MIN,DEN
2,L2M-DEN-MIN-11-03-16,Q4,,Turnover: Palming,Emmanuel Mudiay,,INC,DEN,NaN
3,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,INC,MIN,DEN
4,L2M-DEN-MIN-11-03-16,Q4,,Foul: Loose Ball,Jusuf Nurkic,Nemanja Bjelica,CNC,DEN,MIN
5,L2M-DEN-MIN-11-03-16,Q4,,Instant Replay: Support Ruling,,,CC,NaN,NaN
6,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Tyus Jones,Jameer Nelson,CNC,MIN,DEN
7,L2M-DEN-MIN-11-03-16,Q4,,Foul: Loose Ball,Karl-Anthony Towns,Jusuf Nurkic,CNC,MIN,DEN
8,L2M-DEN-MIN-11-03-16,Q4,,Turnover: 3 Second Violation,Karl-Anthony Towns,,INC,MIN,NaN
9,L2M-DEN-MIN-11-03-16,Q4,,Foul: Personal,Nemanja Bjelica,Jameer Nelson,INC,MIN,DEN


In [22]:
df[['report', '']]

SyntaxError: invalid syntax (<ipython-input-22-1a883b58841f>, line 1)

In [25]:
df[calls.columns]

,report,period,time,call,committing,disadvantaged,decision,committing_team,disadvantaged_team
0,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,CC,MIN,DEN
1,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Karl-Anthony Towns,Danilo Gallinari,CNC,MIN,DEN
2,L2M-DEN-MIN-11-03-16,Q4,,Turnover: Palming,Emmanuel Mudiay,,INC,DEN,NaN
3,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,INC,MIN,DEN
4,L2M-DEN-MIN-11-03-16,Q4,,Foul: Loose Ball,Jusuf Nurkic,Nemanja Bjelica,CNC,DEN,MIN
5,L2M-DEN-MIN-11-03-16,Q4,,Instant Replay: Support Ruling,,,CC,NaN,NaN
6,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Tyus Jones,Jameer Nelson,CNC,MIN,DEN
7,L2M-DEN-MIN-11-03-16,Q4,,Foul: Loose Ball,Karl-Anthony Towns,Jusuf Nurkic,CNC,MIN,DEN
8,L2M-DEN-MIN-11-03-16,Q4,,Turnover: 3 Second Violation,Karl-Anthony Towns,,INC,MIN,NaN
9,L2M-DEN-MIN-11-03-16,Q4,,Foul: Personal,Nemanja Bjelica,Jameer Nelson,INC,MIN,DEN
